In [3]:
import json
import plotly.express as px
import os

In [7]:
results_folder = 'results/2020_05_07/'
gurobi_folder = 'Gurobi/'
ortools_folder = 'OrTools/'
extension1 = 'Extension1/'
extension2 = 'Extension2/'
extension3 = 'Extension3/'
extension4 = 'Extension4/'

gurobi_ext1_folder = results_folder + gurobi_folder + extension1
gurobi_ext2_folder = results_folder + gurobi_folder + extension2
gurobi_ext3_folder = results_folder + gurobi_folder + extension3
gurobi_ext4_folder = results_folder + gurobi_folder + extension4

ortools_ext1_folder = results_folder + ortools_folder + extension1
ortools_ext2_folder = results_folder + ortools_folder + extension2
ortools_ext3_folder = results_folder + ortools_folder + extension3
ortools_ext4_folder = results_folder + ortools_folder + extension4

gurobi_ext1_files = [f for f in os.listdir(gurobi_ext1_folder) if  f.endswith('.json')]
gurobi_ext2_files = [f for f in os.listdir(gurobi_ext2_folder) if f.endswith('.json')]
gurobi_ext3_files = [f for f in os.listdir(gurobi_ext3_folder) if f.endswith('.json')]
gurobi_ext4_files = [f for f in os.listdir(gurobi_ext4_folder) if  f.endswith('.json')]

ortools_ext1_files = [f for f in os.listdir(ortools_ext1_folder) if f.endswith('.json')]
ortools_ext2_files = [f for f in os.listdir(ortools_ext2_folder) if f.endswith('.json')]
ortools_ext3_files = [f for f in os.listdir(ortools_ext3_folder) if  f.endswith('.json')]
ortools_ext4_files = [f for f in os.listdir(ortools_ext4_folder) if f.endswith('.json')]

print(gurobi_ext3_files)

['hole6_gurobi.json', 'dubois20_gurobi.json', 'aim-100-1_6-no-1_gurobi.json', 'dubois22_gurobi.json', 'dubois21_gurobi.json', 'par8-1-c_gurobi.json']


In [ ]:
for file in gurobi_ext3_files:
    

fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada')
fig.show()